# MedCellTypes - YOLO Batch Inference

## Batch Inference (Without Model Serving)

In [0]:
%pip install -U ultralytics==8.3.40 opencv-python==4.10.0.84 ray==2.39.0
# %pip install -U opencv-python==4.10.0.84 ray==2.39.0 git+https://github.com/BlitzBricksterYY-db/ultralytics.git@main
dbutils.library.restartPython()

## Both model flavour A and B will work
1. A. Pytorch flavour model logging and loading
2. B. Pyfunc custom wrapper model logging and loading

In [0]:
# load model back
import mlflow
import ultralytics

model_uri = 'runs:/7d7584ec603e4994b746582c64ca76aa/model' # this is pytorch flavour model
# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
test_path = "/Volumes/mmt_mlops_demos/cv/data/Nuclei_Instance_Dataset/yolo_dataset/val/images/"
input_data = [test_path + f for f in ["human_liver_39.png", "human_melanoma_05.png", "mouse_liver_35.png", "mouse_spleen_02.png"]]
print("We will inference on the following images:", input_data)
model = mlflow.pytorch.load_model(model_uri) # for pytorch flavour model, load using mlflow.pytorch
results = model(input_data, stream=False) # enable stream produces a generator instead of a list. a List consumes more memory.

# #: (Don't Run this cell, it will fail). The below is not working for image data as 'input_data,' as it Failed to deserialize input string data to JSON. 
# # Verify the model with the provided input data using the logged dependencies.
# # For more details, refer to:
# # https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
# mlflow.models.predict(
#     model_uri=model_uri,
#     input_data=input_data,
#     env_manager="uv",
# )

In [0]:
# load model back
import mlflow
import ultralytics

model_uri = 'runs:/4a8cef6f866d414096370c4282dbf55f/model' # this is pyfunc customized model
# Replace INPUT_EXAMPLE with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
test_path = "/Volumes/mmt_mlops_demos/cv/data/Nuclei_Instance_Dataset/yolo_dataset/val/images/"
input_data = [test_path + f for f in ["human_liver_39.png", "human_melanoma_05.png", "mouse_liver_35.png", "mouse_spleen_02.png"]]
print("We will inference on the following images:", input_data)
model = mlflow.pyfunc.load_model(model_uri) # for pytorch flavour model, load using mlflow.pytorch
results = model.unwrap_python_model().model(input_data, stream = False) # enable stream produces a generator instead of a list. a List consumes more memory.

# #: (Don't Run this cell, it will fail). The below is not working for image data as 'input_data,' as it Failed to deserialize input string data to JSON. 
# # Verify the model with the provided input data using the logged dependencies.
# # For more details, refer to:
# # https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
# mlflow.models.predict(
#     model_uri=model_uri,
#     input_data=input_data,
#     env_manager="uv",
# )

In [0]:
import cv2
import matplotlib.pyplot as plt
import math

nrow = math.ceil(len(input_data) / 2)

fig, axs = plt.subplots(nrow, 2, figsize=(10, 10))
for ax, img_loc in zip(axs.flatten(), input_data):
    img = cv2.imread(img_loc)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    ax.set_title(img_loc.split("/")[-1])
    ax.axis('off')

plt.tight_layout()

# display(fig)

In [0]:
fig, axs = plt.subplots(nrow, 2, figsize=(10, 10))
for ax, result in zip(axs.flatten(), results):
    img_w_predictions = result.plot()
    img_rgb = cv2.cvtColor(img_w_predictions, cv2.COLOR_BGR2RGB)
    ax.imshow(img_rgb)
    ax.set_title(result.path.split("/")[-1])
    ax.axis('off')

plt.tight_layout()

In [0]:
len(results)

In [0]:
fig, axs = plt.subplots(len(results), 2, figsize=(10, 20)) #, gridspec_kw={'wspace': 0.01, 'hspace': 0.01})
for ax, result in zip(axs, results):
    # for the same row, original figure vs. original+overlay from prediction figure
    ax_L = ax[0] # left subplot
    ax_R = ax[1] # right subplot

    img_raw = result.orig_img
    img_rgb = cv2.cvtColor(img_raw, cv2.COLOR_BGR2RGB)
    ax_L.imshow(img_rgb)
    ax_L.set_title(result.path.split("/")[-1])
    ax_L.axis('off')

    img_w_predictions = result.plot()
    img_rgb = cv2.cvtColor(img_w_predictions, cv2.COLOR_BGR2RGB)
    ax_R.imshow(img_rgb)
    ax_R.set_title(result.path.split("/")[-1])
    ax_R.axis('off')

plt.tight_layout()
# plt.subplots_adjust(wspace=0.1, hspace=0.1)

In [0]:
# View results
for r in results:
    print(r.boxes[:1])
    print(r.masks[:1])